<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/Analitics/part_01/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

18750.0